In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

# core

> Fill in a module description here

# Load packages

Package requirements are stored in `requirements.yml`.

In [ ]:
from pathlib import Path

# Variables

BNC2014 needs to be downloaded for this script to work. It can be obtained from the official [BNC website](http://corpora.lancs.ac.uk/bnc2014/). 

The following variables need to be updated to the corpus' local path. In the current setting the BNC2014 data were stored in the project folder in the folder `data`, so relative paths were used.

In [ ]:
path_bnc = Path('../data/bnc-2014-spoken')
path_tagged = Path(path_bnc / 'spoken' / 'tagged')
path_metadata = Path(path_bnc / 'spoken' / 'metadata')

In [ ]:
assert path_bnc.exists()
assert path_tagged.exists()
assert path_metadata.exists()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()